# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [13]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name).to("mps")
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

In [ ]:
# Load a sample dataset
from datasets import load_dataset
import os

# TODO: define your dataset and config using the path and name parameters
ds_smoltalk = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")
ds_bigcode = ds_bigcode = load_dataset(path="bigcode/the-stack-smol", data_dir="data/python", token=os.environ["HF_TOKEN"])
ds_jade_gh = load_dataset("lwoollett/jade_github_code", token=os.environ["HF_TOKEN"])

In [32]:
def process_jade_dataset(sample):
    messages = [
        {"role": "user", "content": sample["question"]},
        {"role": "assistant","content": sample["answer"]},
    ]
    return {'messages': messages}

# JADE Dataset is somewhat in chat format, with [...meta, question, answer]
# But for the fun of it we'll convert to messages format
ds_jade_gh = ds_jade_gh.map(process_jade_dataset)


Map: 100%|██████████| 3845/3845 [00:00<00:00, 25967.97 examples/s]


In [ ]:
# Added a quick dataset switcher
# ds = ds_smoltalk
# ds = ds_bigcode
ds = ds_jade_gh

# Set up dataset specific training parameters
dataset_text_field = "content" if ds == ds_bigcode else None
eval_dataset=ds["test"] if ds != ds_bigcode else ds["train"]
# Eval dataset does not exist for bigcode so just shove it back on the training set
max_steps = 1000 if ds != ds_jade_gh else None
num_train_epochs = 1 if ds == ds_jade_gh else None


## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [ ]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=100,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=True,
    dataset_text_field=dataset_text_field,
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

/Users/lukewoollett/repos/smol-course/.conda/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/lukewoollett/repos/smol-course/.conda/lib/python3.11/site-packages/transformers/training_args.py:2248: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(
/Users/lukewoollett/repos/smol-course/.conda/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Map: 100%|██████████| 10000/10000 [00:13<00:00, 720.97 examples/s]
max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [40]:
# Train the model
trainer.train()

# Save the model
trainer.save_model("./sft_output")

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.